<a href="https://colab.research.google.com/github/Rehzende/sorocabatransparente/blob/main/notebooks/extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install "camelot-py[cv]" 
%pip install camelot-py[base]
%pip install "PyPDF2<3.0"
%pip install pillow
%pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\rehze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\rehze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\rehze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\rehze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\rehze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import json
from PIL import Image, ImageDraw, ImageFont
import camelot
import os
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from unidecode import unidecode
import urllib.request
from operator import index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

In [13]:


def put_expenses_values(template, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total):
     fonte = ImageFont.truetype("arial.ttf", 80)
     draw = ImageDraw.Draw(template)
     draw.text((1100, 1060), f" R$ {aluguel_impressora}", font=fonte, fill="#fff")
     draw.text((1100, 1260), f" R$ {combustivel}", font=fonte, fill="#fff")
     draw.text((1100, 1460), f" R$ {mat_escritorio}", font=fonte, fill="#fff")
     draw.text((1100, 1660), f" R$ {postagem}", font=fonte, fill="#fff")
     draw.text((1100, 1850), f" R$ {total}", font=fonte, fill="#000")
     draw.text((1100, 2060), f" R$ {reembolso}", font=fonte, fill="#fff")
     return template

def put_photo_vereador(img, image_name):
    photo = Image.open(image_name)
    photo = photo.resize((947, 1433), Image.LANCZOS )
    posicao_photo = (1800, 800)
    img.paste(photo, posicao_photo)
    return img

def process_images_reports(nome_vereador, image_name, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total, year, month):
    fonte_nome = ImageFont.truetype("assets/arial.ttf", 80)
    img = Image.open("assets/image_template.jpg")
    img = put_photo_vereador(img, image_name)
    draw = ImageDraw.Draw(img)
    draw.text((140, 800), nome_vereador, font=fonte_nome, fill="#fff")   
    draw.text((420, 530), f"{month}/{year}", font=ImageFont.truetype("assets/montserrat.ttf", 120), fill="#fff")   
    img = put_expenses_values(img, aluguel_impressora, combustivel, mat_escritorio, postagem, reembolso, total)
    
    file_name = nome_vereador.lower().replace(" ", "_")
    if not os.path.exists(f"../dados/processados/{year}/{month}/"):
              os.makedirs(f"../dados/processados/{year}/{month}/")
    img.save(f"../dados/processados/{year}/{month}/{file_name}.jpg", 'JPEG')
     


In [14]:
## Mudar para obter a imagem dependendo do ano e se o vereador está na lista de despesas
# import urllib.request
# from unidecode import unidecode

# with open("../dados/vereadores/vereadores.json", "r", encoding="utf8") as f:
#     dados_vereadores = json.load(f)

# for vereador in dados_vereadores:
#     url = vereador["imagem_url"]
#     imagem_vereador = unidecode(vereador['nome'].lower().replace(" ", "_"))
#     nome_arquivo = f"../dados/vereadores/imagens/{imagem_vereador}.jpg"  # nome do arquivo a ser salvo
#     urllib.request.urlretrieve(url, nome_arquivo)

In [15]:
LINKS_FILE_PATH = '../dados/relatorios/links.json'
VEREADORES_FILES_PATH = '../dados/vereadores/vereadores.json'


def main(year_data):
    if not year_data["processed"]:
        for m in year_data['months']:
            try:
                year_processed = True
                if not m["processed"]:
                    year = unidecode(year_data['year'].lower())
                    month = unidecode(m['month'])
                    json_month_report = m['report_path']
                    

                    with open(json_month_report, encoding="utf8") as f:
                        vereadores = json.load(f)
                    
                    for _, valores in vereadores.items():
                        imagem_vereador = unidecode(valores['Vereador'].lower().replace(" ", "_"))
                        process_images_reports(valores["Vereador"], f"../dados/vereadores/imagens/{imagem_vereador}.jpg", valores["Aluguel Impressora"], valores["Material Escrit\u00f3rio"], valores["Postagem"], valores["Combustivel"], valores["Reembolso"], valores["Total"], year, month)
                        m['processed'] = True
                    print(
                        f"O Relatório de {month}/{year} foi processado com sucesso.")
                    
            except Exception as e:
                error = {
                    "message": f"Erro ao processar o arquivo JSON: ",
                    "error":   f"{str(e)}"
                }
                print(error)
                year_processed = False
        year_data['processed'] =  year_processed
        print(
            f"Os Json's do Ano {year_data['year'] } foram processado com sucesso.")

try:
    links_file_path = Path(LINKS_FILE_PATH)
    with links_file_path.open("r", encoding="utf8") as f:
        dados = json.load(f)
except Exception as e:
    print(f"Erro ao ler arquivo JSON: {str(e)}")
    dados = []

for year_data in dados:

    main(year_data)

    with links_file_path.open("w", encoding="utf8") as f:
        json.dump(dados, f, ensure_ascii=False, indent=4)
